# 05. Введение в машинное обучение

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Разделение-данных-на-несколько-выборок" data-toc-modified-id="Разделение-данных-на-несколько-выборок-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на несколько выборок</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li><li><span><a href="#Проверка-моделей-на-адекватность" data-toc-modified-id="Проверка-моделей-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка моделей на адекватность</a></span></li></ul></div>

In [1]:
# подключаем необходимые библиотеки и функции
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Изучение данных

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В таблице содержится следующая информация:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Анализируя данные из таблицы понимаем, что столбец `is_ultra` является целевым признаком, который необходимо предсказать с помощью построенной модели. Все остальные столбцы - признаки объектов.

Выделим целевой признак и поместим его в переменную 'target', остальные признаки - в 'features'.

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

## Разделение данных на несколько выборок

Поскольку тестовой выборки нет, поэтому данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов сделаем равными - по 20% от общего размера выборки. Таким образом, разобьем исходные данные в соотношении 60%:20%:20%.

In [6]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(features, 
                                                                                        target, 
                                                                                        test_size=0.2, 
                                                                                        random_state=12345)

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(features_train_valid, 
                                                                              target_train_valid, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

Проверим выполненные разделения, посмотрим на размеры полученных выборок.

In [8]:
print('Размер обучающей выборки:', features_train.shape)
print('Размер валидирующей выборки:', features_valid.shape)
print('Размер тестовой выборки:', features_test.shape)

Размер обучающей выборки: (1928, 4)
Размер валидирующей выборки: (643, 4)
Размер тестовой выборки: (643, 4)


## Исследование моделей

Исследуем качество разных моделей, меняя гиперпараметры. 

**Модель решающего дерева**

Найдем модель с наибольшим значением точности. Для этого необходимо найти наилучшие значения следующих гиперпараметров:
- 'max_depth' - максимальная глубина дерева (будем перебирать значения от 1 до 10),
- 'min_samples_split' - наименьшее количество объектов выборки, попадающих в узел решающего дерева (от 2 до 10),
- 'min_samples_leaf' - наименьшее количество объектов выборки, необходимых для создания листа решающего дерева (от 1 до 10).

In [9]:
i = 0
tree_df = pd.DataFrame(columns=['max_depth', 'min_samples_split', 'min_samples_leaf', 'accuracy_train', 'accuracy_valid'])
best_model_tree = None
best_result = 0
best_depth = 0
best_split = 0
best_leaf = 0
for depth in tqdm(range(1,11)):
    for split in range(2,11):
        for leaf in range(1,11):
            model = DecisionTreeClassifier(
                random_state=12345, 
                max_depth=depth, 
                min_samples_split=split, 
                min_samples_leaf=leaf
            )
            model.fit(features_train, target_train)
            result_train = model.score(features_train, target_train)
            result_valid = model.score(features_valid, target_valid)
            tree_df.loc[i] = np.array([depth, split, leaf, result_train, result_valid])
            i += 1
            if result_valid > best_result:
                best_model_tree = model
                best_result = result_valid
                best_depth = depth
                best_split = split
                best_leaf = leaf

print("Точность наилучшей модели решающего дерева на валидационной выборке:", best_result)
print("Глубина дерева наилучшей модели решающего дерева на валидационной выборке:", best_depth)
print("min_samples_split наилучшей модели решающего дерева на валидационной выборке:", best_split)
print("min_samples_leaf наилучшей модели решающего дерева на валидационной выборке:", best_leaf)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.84it/s]

Точность наилучшей модели решающего дерева на валидационной выборке: 0.7916018662519441
Глубина дерева наилучшей модели решающего дерева на валидационной выборке: 7
min_samples_split наилучшей модели решающего дерева на валидационной выборке: 2
min_samples_leaf наилучшей модели решающего дерева на валидационной выборке: 8


In [10]:
tree_df.sort_values(by='accuracy_train', ascending=False).head()

,max_depth,min_samples_split,min_samples_leaf,accuracy_train,accuracy_valid
810,10.0,2.0,1.0,0.891079,0.771384
820,10.0,3.0,1.0,0.887967,0.763608
830,10.0,4.0,1.0,0.885892,0.768274
840,10.0,5.0,1.0,0.884855,0.771384
831,10.0,4.0,2.0,0.883817,0.765163


In [11]:
tree_df.sort_values(by='accuracy_valid', ascending=False).head(10)

,max_depth,min_samples_split,min_samples_leaf,accuracy_train,accuracy_valid
607,7.0,8.0,8.0,0.839730,0.791602
577,7.0,5.0,8.0,0.839730,0.791602
587,7.0,6.0,8.0,0.839730,0.791602
547,7.0,2.0,8.0,0.839730,0.791602
627,7.0,10.0,8.0,0.839730,0.791602
617,7.0,9.0,8.0,0.839730,0.791602
557,7.0,3.0,8.0,0.839730,0.791602
597,7.0,7.0,8.0,0.839730,0.791602
567,7.0,4.0,8.0,0.839730,0.791602
727,9.0,2.0,8.0,0.850104,0.790047


**Вывод**

1. Среди результатов достаточно большое количество моделей, которые показали высокую точность на обучающей выборке (более 0.88), но на валидационной выборке - результат сильно меньше (0.76-0.77), что говорит о переобучении этих моделей. На тестовых данных также будет показан невысокий результат.
2. Среди обученных 900 моделей с различными значениями гиперпараметров, 9 моделей показали наибольший результат точности на валидационной выборке - 0.7916. Вместе с этим, данные модели характеризуются невысоким переобучением, т.к. значения точности на обучающей выборке несильно больше - 0.8397. Можем предположить, что на тестовой выборке, результат точности будет также в окрестности 0.79.
3. Среди всех моделей выбрали модель со следующими параметрами:
    - max_depth - 7, 
    - min_samples_split - 2, 
    - min_samples_leaf - 8.

**Модель случайного леса**

Найдем модель с наибольшим значением точности. Для этого необходимо найти наилучшие значения следующих гиперпараметров:
- 'max_depth' - максимальная глубина дерева (будем перебирать значения от 1 до 5),
- 'min_samples_split' - наименьшее количество объектов выборки, попадающих в узел решающего дерева (от 2 до 5),
- 'min_samples_leaf' - наименьшее количество объектов выборки, необходимых для создания листа решающего дерева (от 1 до 5),
- 'n_estimators' - количество деревьев в лесу (от 10 до 50 с шагом 10).

In [12]:
i = 0
forest_df = pd.DataFrame(columns=['n_estimators', 
                                  'max_depth', 
                                  'min_samples_split', 
                                  'min_samples_leaf', 
                                  'accuracy_train', 
                                  'accuracy_valid']
                        )
best_result = 0
best_model_forest = None
best_depth = 0
best_split = 0
best_leaf = 0
best_est = 0
for depth in tqdm(range(1,6)):
    for split in range(2,6):
        for leaf in range(1,6):
            for est in range(10, 51, 10):
                model = RandomForestClassifier(
                    random_state=12345, 
                    n_estimators=est, 
                    max_depth=depth, 
                    min_samples_split=split,
                    min_samples_leaf=leaf
                )
                model.fit(features_train, target_train)
                result_train = model.score(features_train, target_train)
                result_valid = model.score(features_valid, target_valid)
                forest_df.loc[i] = np.array([est, depth, split, leaf, result_train, result_valid])
                i += 1
                if result_valid > best_result:
                    best_model_forest = model
                    best_result = result_valid
                    best_depth = depth
                    best_split = split
                    best_leaf = leaf
                    best_est = est

print("Точность наилучшей модели случайного леса на валидационной выборке:", best_result)
print("Количество деревьев в лесу наилучшей модели случайного леса на валидационной выборке:", best_est)
print("Глубина дерева наилучшей модели случайного леса на валидационной выборке:", best_depth)
print("min_samples_split наилучшей модели случайного леса на валидационной выборке:", best_split)
print("min_samples_leaf наилучшей модели случайного леса на валидационной выборке:", best_leaf)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:27<00:00,  5.50s/it]

Точность наилучшей модели случайного леса на валидационной выборке: 0.7838258164852255
Количество деревьев в лесу наилучшей модели случайного леса на валидационной выборке: 30
Глубина дерева наилучшей модели случайного леса на валидационной выборке: 5
min_samples_split наилучшей модели случайного леса на валидационной выборке: 2
min_samples_leaf наилучшей модели случайного леса на валидационной выборке: 5


In [13]:
forest_df.sort_values(by='accuracy_train', ascending=False).head()

,n_estimators,max_depth,min_samples_split,min_samples_leaf,accuracy_train,accuracy_valid
476,20.0,5.0,5.0,1.0,0.834544,0.782271
454,50.0,5.0,4.0,1.0,0.834025,0.782271
453,40.0,5.0,4.0,1.0,0.834025,0.780715
429,50.0,5.0,3.0,1.0,0.833506,0.780715
428,40.0,5.0,3.0,1.0,0.833506,0.780715


In [14]:
forest_df.sort_values(by='accuracy_valid', ascending=False).head(10)

,n_estimators,max_depth,min_samples_split,min_samples_leaf,accuracy_train,accuracy_valid
473,40.0,5.0,4.0,5.0,0.826763,0.783826
498,40.0,5.0,5.0,5.0,0.826763,0.783826
497,30.0,5.0,5.0,5.0,0.826245,0.783826
448,40.0,5.0,3.0,5.0,0.826763,0.783826
447,30.0,5.0,3.0,5.0,0.826245,0.783826
423,40.0,5.0,2.0,5.0,0.826763,0.783826
472,30.0,5.0,4.0,5.0,0.826245,0.783826
422,30.0,5.0,2.0,5.0,0.826245,0.783826
499,50.0,5.0,5.0,5.0,0.829357,0.782271
477,30.0,5.0,5.0,1.0,0.832988,0.782271


**Вывод**

1. Ситуация с моделью случайного леса отличается от предыдущего случая. Из-за того, что при обучении моделей диапазоны гиперпараметров мы взяли меньшие, чем для моделей решающего дерева, поэтому получили меньшие значения точности по обучающей выборке. Вместе с этим, наибольшие значения точности по валидационной выборке получились примерно такие же, как и для решающего дерева - 0.78. 
2. Среди всех моделей выбрали модель со следующими параметрами:
    - n_estimators = 30,
    - max_depth - 5, 
    - min_samples_split - 2, 
    - min_samples_leaf - 5.

**Логистическая регрессия**

Обучим модель логистической регрессии.

In [15]:
model_log_regr = LogisticRegression(random_state=12345)
model_log_regr.fit(features_train, target_train)
result = model_log_regr.score(features_valid, target_valid)

print("Точность модели на валидационной выборке:", result)

Точность модели на валидационной выборке: 0.7262830482115086


**Вывод**

Получили сравнительно низкую точность модели логистической регрессии. Так как по условию необходимо добиться точности не менее 0.75 на тестовой выборке, предполагаем, что данная модель не способна достичь требуемого результата. В дальнейшем использовать ее не будем.

## Проверка моделей на тестовой выборке

В предыдущем шаге мы выделили 2 модели, которые могут выдать требуемую точность для тестовой выборки. Проверим их.

In [16]:
# Проверка выбранной модели решающего дерева
result_test = best_model_tree.score(features_test, target_test)
print("Accuracy модели на тестовой выборке:", result_test)

Accuracy модели на тестовой выборке: 0.7931570762052877


In [17]:
# Проверка выбранной модели случайного леса
result_test = best_model_forest.score(features_test, target_test)
print("Accuracy модели на тестовой выборке:", result_test)

Accuracy модели на тестовой выборке: 0.7962674961119751


**Вывод**

Обе модели показали примерно одинаковые значения точности на тестовой выборке - 0.79. Данное значение удовлетворяет заданному в условии требованию к точности (не менее 0.75). Какую из них выбрать для заказчика? - Однозначно пока сказать нельзя, необходимо проверить их на адекватность.

## Проверка моделей на адекватность

Для того чтобы проверить модель на вменяемость, необходимо сравнить ее со случайной моделью.

In [18]:
len(target)

3214

In [19]:
target.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Всего в исходной таблице 3214 строк, в которых значение '1' в около 30% случаев. Поэтому, случайной моделью, с которой мы будем сравнивать, является такая модель, которая в 7 случаях из 10 выдает значение '0', а в 3 случаях из 10 - значение '1'.

In [20]:
list_model1 = [0]*7 + [1]*3
list_model1

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

В список 'list1' случайным образом помещаются элементы '0' или '1' из списка 'list_model1'. Затем полученный вектор сравнивается с исходными целевыми значениями 'target' и считается точность. Данная процедура выполняется 10000 раз.

In [21]:
list1 = []
list2 = []
for i in tqdm(range(10000)):
    for _ in range(3214):
        list1.append(random.choice(list_model1))
    list2.append(accuracy_score(target, list1))
    list1=[]
random_ac = pd.Series(list2)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:40<00:00, 247.95it/s]


In [22]:
random_ac.mean()

0.577387181082763

Получили среднюю точность для такой случайной модели ~0.58.

Однако можем использовать другую случайную модель для проверки. Например такую: константная модель, которая всегда возвращает '0' (list_model2).

In [23]:
list_model2 = [0]*3214
list_model2[:5]

[0, 0, 0, 0, 0]

Посмотрим на ее качество (точность):

In [24]:
accuracy_score(target, list_model2)

0.693528313627878

Случайная модель более простая, чем в первом случае, но точность ее ~0.7.

**Вывод**

Таким образом, проверили обученные модели решающего дерева и случайного леса на адекватность. Выяснили, что их точность незначительно превышает точность полностью случайной модели, следовательно не следует полностью надеяться на них при анализе поведения клиентов.